In [ ]:
import flopy
import os
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import os
import matplotlib.pyplot as plt



In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params()

datestart = info['start_date']

name = info['name']

out_folder = basic.out_folder()
print(datestart)

In [ ]:
m = basic.load_model()

In [ ]:
swr = gpd.read_file("GIS/SWR_Reaches.shp")

In [ ]:
srt = m.bas6.strt.array[0]
top = m.dis.top.array
bot = m.dis.botm.array[0]
plt.imshow(srt)

In [ ]:
swr.loc[:,'starting_heads'] = srt[swr.loc[:,'i'], swr.loc[:,'j']]
swr.loc[:,'top'] = top[swr.loc[:,'i'], swr.loc[:,'j']]
swr.loc[:,'bottom'] = bot[swr.loc[:,'i'], swr.loc[:,'j']]
hds, hdsall = basic.get_heads(m)

swr.loc[:,'final_head'] = hds[0][swr.loc[:,'i'], swr.loc[:,'j']]

In [ ]:
srt_stage = pd.read_csv(os.path.join('RR_2022/inputs/start_stage.tab'), sep = '\t',header = None)
srt_stage.columns = ['rno','start_stage']

if srt_stage.at[0,'rno']==1:
    pass
else:
    srt_stage = srt_stage.loc[1:]
    srt_stage = srt_stage.astype({'rno':int,'start_stage':np.float64})
srt_stage.dtypes

In [ ]:
min_elev = pd.read_csv(os.path.join('RR_2022/inputs/stream_thalwegs.txt')).set_index('reach')
min_elev.head(1)

In [ ]:
ax = pd.DataFrame(swr.set_index('rno').loc[:,['starting_heads']]).plot(figsize = (10,10))
ax = pd.DataFrame(srt_stage.set_index('rno').loc[:,['start_stage']]).plot(ax = ax, color = 'green', ls = '-',lw = 3)
ax = pd.DataFrame(swr.set_index('rno').loc[:,['top']]).plot(ax = ax)
ax = pd.DataFrame(swr.set_index('rno').loc[:,['bottom']]).plot(ax = ax)
ax = pd.DataFrame(swr.set_index('rno').loc[:,['final_head']]).plot(ax = ax, color = 'k',lw = 3)
min_elev.plot(ax = ax, color = 'r', ls = '--',lw = 3)

plt.savefig(os.path.join(out_folder, 'star_stage.png'), dpi = 250)